In [3]:
# Update dependencies first
!pip install -U bitsandbytes accelerate transformers peft trl datasets wandb nlpaug

In [4]:
# Import necessary libraries
from pathlib import Path
import os
import random
import numpy as np
import pandas as pd
import torch
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch.nn as nn

# Verify GPU and CUDA setup
!nvidia-smi
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

base_folder = Path('/content/drive/MyDrive/data')
data_folder = Path('/content/inclass_kaggle_data')
kaggle_api = base_folder/'.kaggle'
model_folder = base_folder/'models/nlp_spring_2025/inclass_kaggle/Qwen3'
archive_folder = data_folder/'archive'

os.environ['KAGGLE_CONFIG_DIR'] = str(kaggle_api)
!chmod 600 "{kaggle_api}/kaggle.json"

data_folder.mkdir(exist_ok=True, parents=True)
kaggle_api.mkdir(exist_ok=True, parents=True)
model_folder.mkdir(exist_ok=True, parents=True)
archive_folder.mkdir(exist_ok=True, parents=True)

Fri May  9 04:16:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             46W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
import gc
import torch
import time

def free_gpu_memory():
    """
    Frees up GPU memory after CUDA out-of-memory error in Colab.
    """
    try:
        # Delete all torch tensors to free up memory
        for obj in list(locals().values()):
            if torch.is_tensor(obj):
                del obj

        # Collect garbage to release any remaining unused memory
        gc.collect()

        # Empty the CUDA cache to release GPU memory
        torch.cuda.empty_cache()

        # Adding a small delay to allow memory to be fully released
        time.sleep(2)

        print("✅ GPU memory has been freed.")
    except Exception as e:
        print(f"❌ Error while freeing GPU memory: {e}")


In [6]:
!kaggle competitions download emotion-detection-spring-2025 -p {archive_folder}

import zipfile
with zipfile.ZipFile(archive_folder / "emotion-detection-spring-2025.zip", 'r') as zip_ref:
    zip_ref.extractall(data_folder)

In [7]:
import pandas as pd

train_df = pd.read_csv(data_folder / 'train.csv')
test_df = pd.read_csv(data_folder / 'test.csv')
sample_submission = pd.read_csv(data_folder / 'sample_submission.csv')

# Text preprocessing function
def preprocess_text(df):
    """Clean and prepare text data"""
    df = df.copy()
    df['Tweet'] = df['Tweet'].str.replace(r'http\S+', '', regex=True)  # Remove URLs
    df['Tweet'] = df['Tweet'].str.replace(r'@\w+', '@user', regex=True)  # Normalize mentions
    df['Tweet'] = df['Tweet'].str.replace(r'#(\w+)', r'\1', regex=True)  # Remove # but keep hashtag text
    return df

# Apply preprocessing
train_df = preprocess_text(train_df)
test_df = preprocess_text(test_df)

# Define emotion label columns
label_cols = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
              'optimism', 'pessimism', 'sadness', 'surprise', 'trust']


In [8]:
# Data Augmentation for minority classes
def augment_data(df, target_labels, aug_multiplier=1.5):
    """Augment data for minority classes to address class imbalance"""
    try:
        import nlpaug.augmenter.word as naw

        # Create synonym replacement augmenter
        aug_synonym = naw.SynonymAug(aug_src='wordnet')

        # Calculate class distribution
        class_counts = df[target_labels].sum().sort_values()
        minority_labels = class_counts[class_counts < class_counts.median()].index.tolist()

        augmented_rows = []

        # For each minority class, augment examples
        for label in minority_labels:
            # Get samples that have this label
            positive_samples = df[df[label] == 1]

            # Determine how many samples to generate
            num_to_generate = int(len(positive_samples) * (aug_multiplier - 1))

            if num_to_generate > 0:
                # Select samples to augment
                samples_to_augment = positive_samples.sample(
                    n=min(num_to_generate, len(positive_samples)),
                    replace=(num_to_generate > len(positive_samples))
                )

                # Augment each sample
                for _, row in samples_to_augment.iterrows():
                    try:
                        # Augment the text
                        augmented_text = aug_synonym.augment(row['Tweet'])

                        # Create new row with augmented text
                        new_row = row.copy()
                        new_row['Tweet'] = augmented_text
                        augmented_rows.append(new_row)
                    except Exception as e:
                        print(f"Error augmenting text: {e}")
                        continue

        # Combine original and augmented data
        if augmented_rows:
            augmented_df = pd.DataFrame(augmented_rows)
            return pd.concat([df, augmented_df], ignore_index=True)

    except ImportError:
        print("nlpaug not available, skipping augmentation")

    return df

In [9]:
# Augment training data
train_df = augment_data(train_df, label_cols)

# Convert multi-label format to comma-separated labels for generative approach
def format_emotion_labels(row):
    """Convert multi-hot encoding to text labels"""
    present_emotions = [emotion for emotion, value in zip(label_cols, row[label_cols]) if value == 1]
    return ", ".join(present_emotions) if present_emotions else "none"

# Add text label column to the dataframes
train_df['emotion_text'] = train_df.apply(format_emotion_labels, axis=1)
val_df = train_df.sample(frac=0.15, random_state=42)
train_df = train_df.drop(val_df.index)

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is al

Error augmenting text: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load taggers/averaged_perceptron_tagger_eng/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error augmenting text: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  F

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger t

Error augmenting text: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load taggers/averaged_perceptron_tagger_eng/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error augmenting text: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  F

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger t

...

Error augmenting text: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load taggers/averaged_perceptron_tagger_eng/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error augmenting text: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  F

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger t

Error augmenting text: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load taggers/averaged_perceptron_tagger_eng/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error augmenting text: 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  F

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger t

Training samples: 6565
Validation samples: 1159
Test samples: 3259


In [10]:
# Initialize W&B
wandb.init(
    project="emotion_detection_qwen3_generation",
    name=f"Qwen3-0.6B-Label-Generation-{wandb.util.generate_id()}",
    reinit=True
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shreevershith (my-wandb-account) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


In [11]:
# Choose which model to use - Base or Instruction-tuned
use_base_model = True  # Set to False to use the instruction-tuned model

if use_base_model:
    model_name = "Qwen/Qwen3-0.6B-Base"
    print("Using Base Model: Qwen/Qwen3-0.6B-Base")
else:
    model_name = "Qwen/Qwen3-0.6B"
    print("Using Instruction-tuned Model: Qwen/Qwen3-0.6B")

# Try different quantization approaches - if 4-bit fails, fall back to 8-bit or fp16
try:
    # QLoRA 4-bit configuration - first attempt
    print("Attempting 4-bit quantization...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    # Load base model with 4-bit quantization
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    print("4-bit quantization successful!")

except Exception as e:
    print(f"4-bit quantization failed: {e}")
    try:
        # Fall back to 8-bit quantization
        print("Falling back to 8-bit quantization...")
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_quant_type="nf4"
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            quantization_config=bnb_config,
            device_map="auto"
        )
        print("8-bit quantization successful!")

    except Exception as e2:
        print(f"8-bit quantization failed: {e2}")
        print("Falling back to FP16...")

        # Fall back to FP16 (no quantization)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            device_map="auto",
            torch_dtype=torch.float16
        )
        print("Model loaded in FP16!")

Using Base Model: Qwen/Qwen3-0.6B-Base
Attempting 4-bit quantization...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

4-bit quantization successful!


In [12]:
# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA adapter to the model
model = get_peft_model(model, lora_config)
print(f"Trainable parameters: {model.print_trainable_parameters()}")

trainable params: 10,092,544 || all params: 606,142,464 || trainable%: 1.6650
Trainable parameters: None


In [13]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

# Define the classification template for your format_chat function
classification_template = """
{%- for message in messages -%}
    {%- if message['role'] == 'user' -%}
        {{ message['content'] }}
    {%- elif message['role'] == 'assistant' -%}
        ### LABEL: {{ message['content'] }}{{eos_token}}
    {%- endif -%}
{%- endfor -%}
{%- if add_generation_prompt -%}
### LABEL:
{%- endif -%}
"""

# Apply the template to the tokenizer
tokenizer.chat_template = classification_template
print("✅ Tokenizer template is set.")

✅ Tokenizer template is set.


In [14]:
# # Create prompt templates based on model type
# def create_prompt_template(use_base_model):
#     if use_base_model:
#         # Simple prompt for base model
#         def prompt_fn(tweet, emotion_text=None):
#             if emotion_text is not None:
#                 return f"Tweet: {tweet}\nEmotions: {emotion_text}</s>"
#             else:
#                 return f"Tweet: {tweet}\nEmotions:"
#     else:
#         # Instruction format for instruction-tuned model
#         def prompt_fn(tweet, emotion_text=None):
#             if emotion_text is not None:
#                 return f"<|im_start|>user\nIdentify the emotions expressed in this tweet: {tweet}<|im_end|>\n<|im_start|>assistant\n{emotion_text}<|im_end|>"
#             else:
#                 return f"<|im_start|>user\nIdentify the emotions expressed in this tweet: {tweet}<|im_end|>\n<|im_start|>assistant\n"

#     return prompt_fn

# prompt_template = create_prompt_template(use_base_model)

# Create prompt templates based on model type
def create_prompt_template(use_base_model):
    if use_base_model:
        # Simple prompt for base model
        def prompt_fn(tweet, emotion_text=None):
            instruction = f"Classify the following tweet by selecting all applicable labels from the list: {label_cols}. ### TEXT: {tweet}"
            if emotion_text:
                instruction += f"\nEmotions: {emotion_text}"

            messages = [
                {"role": "user", "content": instruction},
                {"role": "assistant", "content": ""}
            ]
            # Apply the chat template for structured input
            return tokenizer.apply_chat_template(messages, tokenize=False, eos_token=tokenizer.eos_token)
    else:
        # Instruction format for instruction-tuned model
        def prompt_fn(tweet, emotion_text=None):
            instruction = f"Identify the emotions expressed in this tweet: {tweet}"
            if emotion_text:
                instruction += f"\nEmotions: {emotion_text}"

            messages = [
                {"role": "user", "content": instruction},
                {"role": "assistant", "content": ""}
            ]
            return tokenizer.apply_chat_template(messages, tokenize=False, eos_token=tokenizer.eos_token)

    return prompt_fn

# Initialize the prompt template
prompt_template = create_prompt_template(use_base_model)
print("✅ Prompt template is set.")

✅ Prompt template is set.


In [15]:
# Custom dataset for generating emotion labels
class EmotionLabelGenerationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, prompt_template, max_length=256, training=True):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.prompt_template = prompt_template
        self.max_length = max_length
        self.training = training

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        tweet = row['Tweet']

        if self.training:
            # For training data, include the emotion text in the prompt
            emotion_text = row['emotion_text']
            prompt = self.prompt_template(tweet, emotion_text)
        else:
            # For testing, just provide the input prompt
            prompt = self.prompt_template(tweet)

        # Tokenize input
        inputs = self.tokenizer(
            prompt,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        item = {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
        }

        if self.training:
            # Create labels for causal language modeling
            item['labels'] = item['input_ids'].clone()

            # Mask out labels for input portion (we only want to predict the output)
            if use_base_model:
                # Find position where "Emotions:" appears
                emotions_pos = (item['input_ids'] == self.tokenizer.encode("Emotions:", add_special_tokens=False)[0]).nonzero(as_tuple=True)[0]
                if len(emotions_pos) > 0:
                    item['labels'][:emotions_pos[-1] + 1] = -100
            else:
                # Find position of the assistant token
                assistant_token = self.tokenizer.encode("<|im_start|>assistant", add_special_tokens=False)[-1]
                assistant_pos = (item['input_ids'] == assistant_token).nonzero(as_tuple=True)[0]
                if len(assistant_pos) > 0:
                    item['labels'][:assistant_pos[-1] + 1] = -100

        return item

In [16]:
# Create datasets
train_dataset = EmotionLabelGenerationDataset(train_df, tokenizer, prompt_template)
val_dataset = EmotionLabelGenerationDataset(val_df, tokenizer, prompt_template)
test_dataset = EmotionLabelGenerationDataset(test_df, tokenizer, prompt_template, training=False)

# Custom data collator that handles the creation of language modeling labels
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [18]:
# Modified compute_metrics function for language generation evaluation
def compute_generation_metrics(eval_preds):
    """
    Compute F1 scores by actually generating text and extracting labels
    rather than trying to use output logits directly
    """
    # We can't directly use the logits/labels as this is a text generation task
    print("Evaluating generation performance...")

    # Extract predictions from generated text
    # We'll need to sample a subset for evaluation as generating all would be slow
    eval_size = min(len(val_dataset), 100)  # Limit to 100 samples for speed

    # Track metrics
    all_true_labels = []
    all_pred_labels = []

    # Sample indices for evaluation
    eval_indices = random.sample(range(len(val_dataset)), eval_size)

    # Generate predictions for the sampled validation examples
    model.eval()
    for idx in eval_indices:
        # Get original sample
        sample = val_df.iloc[idx]
        tweet = sample['Tweet']

        # Get true emotion labels
        true_emotion_labels = [1 if sample[label] == 1 else 0 for label in label_cols]
        all_true_labels.append(true_emotion_labels)

        # Create prompt and generate text
        prompt = prompt_template(tweet)
        inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=32,
                num_beams=2,
                early_stopping=True
            )

        # Decode generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract emotions from generated text
        emotions = extract_emotions(generated_text, label_cols)

        # Create binary vector
        binary_labels = [1 if label in emotions else 0 for label in label_cols]
        all_pred_labels.append(binary_labels)

    # Convert to numpy arrays
    all_true_labels = np.array(all_true_labels)
    all_pred_labels = np.array(all_pred_labels)

    # Calculate metrics
    f1_macro = f1_score(all_true_labels, all_pred_labels, average="macro", zero_division=0)
    f1_micro = f1_score(all_true_labels, all_pred_labels, average="micro", zero_division=0)
    f1_weighted = f1_score(all_true_labels, all_pred_labels, average="weighted", zero_division=0)
    acc = accuracy_score(all_true_labels, all_pred_labels)

    # Optional: Display classification report for a few classes
    print("\nClassification Report (Sample):")
    print(classification_report(
        all_true_labels,
        all_pred_labels,
        target_names=label_cols[:3],  # Show first 3 classes
        zero_division=0
    ))

    # Log detailed evaluation metrics
    wandb.log({
        "eval/f1_macro": f1_macro,
        "eval/f1_micro": f1_micro,
        "eval/f1_weighted": f1_weighted,
        "eval/accuracy": acc
    })

    # Log class distribution in predictions
    class_dist = all_pred_labels.sum(axis=0)
    for i, label in enumerate(label_cols):
        wandb.log({f"pred_count_{label}": class_dist[i]})

    print(f"F1 Macro: {f1_macro:.4f}, F1 Micro: {f1_micro:.4f}, Accuracy: {acc:.4f}")

    return {
        "f1": f1_macro,
        "f1_macro": f1_macro,
        "f1_micro": f1_micro,
        "accuracy": acc
    }

from transformers import TrainerCallback

# Custom evaluation callback to run after validation steps
class F1EvaluationCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        """Run F1 evaluation after validation"""
        print("\n--- Running custom F1 evaluation ---")
        metrics = compute_generation_metrics(None)  # We don't need the preds parameter

        # Log metrics to wandb with step
        for key, value in metrics.items():
            wandb.log({f"eval/{key}": value}, step=state.global_step)

        # Update best metric if better than previous best
        if state.best_metric is None or metrics["f1_macro"] > state.best_metric:
            state.best_metric = metrics["f1_macro"]
            control.should_save = True
            print(f"New best F1 score: {metrics['f1_macro']:.4f}")

        return control

In [20]:
# Improved training arguments with memory optimization
training_args = TrainingArguments(
    output_dir=f"./qwen3_{use_base_model}_base_model_results",
    # Reduce batch sizes
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    # Increase gradient accumulation to compensate for smaller batches
    gradient_accumulation_steps=16,
    learning_rate=2e-5,
    # Reduce number of epochs for faster training
    num_train_epochs=2,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    # Evaluate and save less frequently to reduce memory spikes
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,         # Changed to True since higher F1 is better
    report_to=["wandb"],
    # Memory optimizations
    fp16=False,                     # Turn off fp16 as it can sometimes cause issues
    bf16=True,                      # Keep bfloat16 which is more stable
    optim="adamw_torch_fused",      # Use fused optimizer for memory efficiency
    remove_unused_columns=False,
    label_names=["labels"],
    # Additional memory optimizations
    gradient_checkpointing=True,    # Enable gradient checkpointing for memory savings
    ddp_find_unused_parameters=False, # Avoid unused parameter check
    max_grad_norm=1.0,              # Clip gradients to prevent training instability
)

# Check GPU memory before training
print(f"GPU memory before training: {torch.cuda.memory_allocated() / 1024**3:.2f}GB / {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f}GB")

# Make sure we've freed memory
free_gpu_memory()

# Define trainer with same parameters
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_generation_metrics
)

# Limit the length of input sequences to save memory
max_sequence_length = 128  # Try a shorter sequence length
train_dataset = EmotionLabelGenerationDataset(train_df, tokenizer, prompt_template, max_length=max_sequence_length)
val_dataset = EmotionLabelGenerationDataset(val_df, tokenizer, prompt_template, max_length=max_sequence_length)

# Free memory again before training
free_gpu_memory()

# Train the model
try:
    print("Starting training...")
    trainer.train()
    print("Training completed successfully!")
except RuntimeError as e:
    print(f"Training error: {e}")
    print("Attempting to recover...")
    free_gpu_memory()

    # If training fails, try with even lower memory settings
    print("Retrying with more aggressive memory optimization...")

    # Save the current state of the model anyway
    trainer.save_model(model_folder / f"qwen3_label_generation_{'base' if use_base_model else 'instruction'}_partial")

    # Create a smaller subsample to debug
    debug_train_df = train_df.sample(frac=0.5, random_state=42)
    debug_train_dataset = EmotionLabelGenerationDataset(debug_train_df, tokenizer, prompt_template, max_length=96)

    # Update trainer with smaller dataset
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            **{**training_args.__dict__,
               "per_device_train_batch_size": 1,
               "gradient_accumulation_steps": 32,
               "max_steps": 500,  # Limit to fewer steps
               "evaluation_strategy": "steps",
               "eval_steps": 100,
               "save_strategy": "steps",
               "save_steps": 100,
            }),
        train_dataset=debug_train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Try training again with reduced settings
    trainer.train()


GPU memory before training: 0.83GB / 39.56GB
✅ GPU memory has been freed.
✅ GPU memory has been freed.
Starting training...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


Training completed successfully!


In [21]:
# Save the trained model
trainer.save_model(model_folder / f"qwen3_label_generation_{'base' if use_base_model else 'instruction'}")

In [22]:
# Function to extract emotion labels from generated text
def extract_emotions(text, label_cols):
    """Extract emotion labels from generated text"""
    text = text.lower().strip()

    # Handle different output formats
    if "emotions:" in text:
        text = text.split("emotions:")[-1].strip()
    elif "<|im_end|>" in text:
        text = text.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()

    # Split by common separators and clean up
    emotions = []
    for sep in [',', ' and ', ';', '\n']:
        if sep in text:
            emotions.extend([e.strip() for e in text.split(sep)])
            break
    else:
        # If no separator found, treat the whole text as a single emotion
        emotions = [text.strip()]

    # Match with valid emotions and remove duplicates
    valid_emotions = []
    for emotion in emotions:
        for label in label_cols:
            if label in emotion:
                valid_emotions.append(label)
                break

    return list(set(valid_emotions))

In [23]:
# Generate predictions for test data
model.eval()
predictions = []

print("Generating predictions...")
for i, sample in enumerate(test_df['Tweet']):
    if i % 100 == 0:
        print(f"Processing sample {i}/{len(test_df)}")

    # Create prompt
    prompt = prompt_template(sample)
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=32,
            num_beams=4,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    # Decode generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract emotions from generated text
    emotions = extract_emotions(generated_text, label_cols)

    # Create binary vector
    binary_labels = [1 if label in emotions else 0 for label in label_cols]
    predictions.append(binary_labels)

    if i < 5:  # Print a few examples
        print(f"Sample: {sample}")
        print(f"Generated: {generated_text}")
        print(f"Extracted emotions: {emotions}")
        print(f"Binary labels: {binary_labels}")
        print("---")

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Generating predictions...
Processing sample 0/3259


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Sample: @user @user Dont worry Indian army is on its ways to dispatch all Terrorists to Hell
Generated: Classify the following tweet by selecting all applicable labels from the list: ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']. ### TEXT: @user @user Dont worry Indian army is on its ways to dispatch all Terrorists to Hell### LABEL: 
Extracted emotions: ['love', 'surprise', 'optimism', 'pessimism', 'fear', 'anger', 'anticipation', 'disgust', 'trust', 'sadness', 'joy']
Binary labels: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
---


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for

Processing sample 3100/3259


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for

Processing sample 3200/3259


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for

In [24]:
# Create submission dataframe
submission = pd.DataFrame(predictions, columns=label_cols)
submission.insert(0, 'ID', test_df['ID'])

# Check if we have any samples without predictions
empty_predictions = submission[label_cols].sum(axis=1) == 0
if empty_predictions.any():
    print(f"Warning: {empty_predictions.sum()} samples have no predicted emotions")
    # For samples with no predictions, add the most common emotion as a fallback
    most_common = train_df[label_cols].sum().idxmax()
    submission.loc[empty_predictions, most_common] = 1

# Save submission
model_type = 'base' if use_base_model else 'instruction'
submission_path = model_folder / f'qwen3_{model_type}_generated_labels_submission.csv'
submission.to_csv(submission_path, index=False)
print('✅ Submission saved:', submission_path)

✅ Submission saved: /content/drive/MyDrive/data/models/nlp_spring_2025/inclass_kaggle/Qwen3/qwen3_base_generated_labels_submission.csv


In [25]:
# Print prediction stats
total_predictions = submission[label_cols].sum().sum()
print(f"Total positive predictions: {total_predictions}")
print("Predictions per class:")
for col in label_cols:
    class_count = submission[col].sum()
    print(f" - {col}: {class_count} ({class_count/len(submission)*100:.2f}%)")

Total positive predictions: 31196
Predictions per class:
 - anger: 3155 (96.81%)
 - anticipation: 2883 (88.46%)
 - disgust: 3160 (96.96%)
 - fear: 3063 (93.99%)
 - joy: 2975 (91.29%)
 - love: 2879 (88.34%)
 - optimism: 3007 (92.27%)
 - pessimism: 2931 (89.94%)
 - sadness: 2975 (91.29%)
 - surprise: 2834 (86.96%)
 - trust: 1334 (40.93%)


In [26]:
# Submit to Kaggle
competition = "emotion-detection-spring-2025"
!kaggle competitions submit -c {competition} -f {submission_path} -m "Qwen3 {model_type} model with direct label generation approach"

100% 105k/105k [00:00<00:00, 284kB/s]
Successfully submitted to Emotion Detection Spring2025

In [27]:
# Finalize wandb run
wandb.finish()

train/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▆▇▇██
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▆▇▇██
train/grad_norm,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▅██▇▇▆▆▅▄▄▄▃▃▂▂▁
train/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,9072216637440000.0
train/epoch,1.9968
train/global_step,820
train/grad_norm,0.94301
train/learning_rate,0.0
train/loss,1.1182
